## T2D disease prediction  

In [1]:
import numpy as np
import pandas as pd 
import os

disease = "T2D" 
# feature_string = 'K_' or 'gi_'
def loadData(feature_string , label_string , label_dict) :
    #read file
    
    filename = "./data/abundance_" + disease + ".txt"  
    if os.path.isfile(filename) :
        rawdata = pd.read_csv(filename , sep = '\t' , index_col=0 , header=None) 
    else :
        print("FileNotFoundError: File {} does not exist".format(filename))
        exit()

    # select rows having feature index identifier string  
    X = rawdata.loc[rawdata.index.str.contains(feature_string, regex=False)].astype('float64')

    # get class labels
    Y = rawdata.loc[label_string] #'disease'
    Y = Y.replace(label_dict).astype('int')
     
    return X , Y 

# def prepare_data(config) :
feature_string = 'k__'
label_string = 'disease'
label_dict = {
    # Controls
    'n': 0,
    # Cirrhosis
    'cirrhosis': 1, 
    # T2D and WT2D
    't2d': 1,
    # Obesity
    'leaness': 0, 'obesity': 1,
}

Raw_X_data , labels = loadData(feature_string , label_string , label_dict )
Raw_X_data = Raw_X_data.transpose() 
labels = labels.values
def filter_data(x , y , filter_thresh) :
    
    classes = np.unique(y) 
    index = x.index.values  

    num_counts = {} 
    for c in classes :
        sub_x = x[y == c]
        num_samples = len(sub_x) 
        # sub_x[sub_x > 0].count()  
        num_counts[str(c)] = sub_x[sub_x > 0].count() / float(num_samples)

    core = pd.DataFrame(index=index)
    for feature in x.columns.values:
        for c in classes : 
            if(num_counts[str(c)].loc[feature] >= filter_thresh) :
                #core[feature] = x[feature].copy()
                core = pd.concat([core , x[feature]] , axis=1)
                break 
    return core 

def get_feature_df(features):
    kingdom, phylum, cl, order, family, genus, species  = [], [], [], [], [], [], []
    for f in features:

        name = f.split("k__")[1].split("|p__")[0].replace(".","")
        if "_unclassified" in name:
            name = 'unclassified_' + name.split("_unclassified")[0]
        kingdom.append(name)

        if "p__" in f:
            name =f.split("p__")[1].split("|c__")[0].replace(".","")
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                phylum.append(name)
            else:
                phylum.append("NA")
        else:
            phylum.append("NA")
            
        if "c__" in f:
            name = f.split("c__")[1].split("|o__")[0].replace(".","")
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                cl.append(name)
            else:
                cl.append("NA")
        else:
            cl.append("NA")
            
        if "o__" in f:
            name = f.split("o__")[1].split("|f__")[0].replace(".","")
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                order.append(name)
            else:
                order.append("NA")
        else:
            order.append("NA")
            
        if "f__" in f:
            name = f.split("f__")[1].split("|g__")[0].replace(".","")
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                family.append(name)
            else:
                family.append("NA")
        else:
            family.append("NA")
            
        if "g__" in f:
            name = f.split("g__")[1].split("|s__")[0].replace(".","")
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                genus.append(name)
            else:
                genus.append("NA")
        else:
            genus.append("NA")
            
        if "s__" in f:
            name = f.split("s__")[1]
            if "_unclassified" in name:
                name = 'unclassified_' + name.split("_unclassified")[0]
            if name != "":
                species.append(name)
            else:
                species.append("NA")
        else:
            species.append("NA")
            
    if len(species) == 0:
        d = {'kingdom': kingdom, 'phylum': phylum, 'class':cl,
            'order':order, 'family':family, 'genus':genus}
        feature_df = pd.DataFrame(data=d)
        feature_df.index = feature_df['genus']
    else:
        d = {'kingdom': kingdom, 'phylum': phylum, 'class':cl,
            'order':order, 'family':family, 'genus':genus, 'species': species}
        feature_df = pd.DataFrame(data=d)
        feature_df.index = feature_df['species']
    return feature_df
 
filter_X_data = filter_data(Raw_X_data , labels , 0.1)
features = list(filter_X_data.columns.values)
features_df = get_feature_df(features)  
print("samples are %d , Raw features are %d ..." % (Raw_X_data.shape[0] ,  Raw_X_data.shape[1]))  
print("filter data after samples are %d , filter Raw features are %d ..." % (filter_X_data.shape[0] ,  filter_X_data.shape[1])) 

samples are 344 , Raw features are 572 ...
filter data after samples are 344 , filter Raw features are 225 ...


In [2]:
classes = set(labels)
for c in classes:
    print(list(labels).count(c)) 

174
170


In [3]:
from graph import Graph
import pickle
from joblib import Parallel, delayed
import multiprocessing
from copy import deepcopy

#Convert abundance vector into tree matrix
def generate_maps(x, g, f, p=-1):
	id = multiprocessing.Process()._identity
	temp_g = deepcopy(g)
	temp_g.populate_graph(f, x)
	map = temp_g.get_map()
	vector = temp_g.graph_vector_features()
	del(temp_g)
	return x, np.array(map), np.array(vector)

def generate_dense_maps(x, g, f, p=-1):
    id = multiprocessing.Process()._identity
    temp_g = deepcopy(g)
    temp_g.populate_graph(f, x)
    map = temp_g.get_dense_map()
    vector = temp_g.graph_vector_features()
    del(temp_g)
    return np.array(map)

print("Contsructing tree..")
g = Graph()
g.build_graph()
g.prune_graph(features_df)
print("Populating trees...")	
results1 = Parallel(n_jobs=1)(delayed(generate_maps)(x,g,features_df) for x in filter_X_data.values)
x_data_sparse_maps = np.array(np.take(results1,1,1).tolist()) 
results2 = Parallel(n_jobs=4)(delayed(generate_dense_maps)(x,g,features_df) for x in filter_X_data.values)
x_data_dense_maps = np.array(results2) 
filter_x_data = filter_X_data.values 

print(x_data_sparse_maps.shape)
print(x_data_dense_maps.shape)
print(filter_x_data.shape)

Contsructing tree..
Pruning Tree...
Populating trees...


c:\Users\70923\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


(344, 10, 159)
(344, 5, 159)
(344, 225)


In [4]:
num_runs = 10
num_test = 10 
num_class = len(np.unique(labels))  

seed = np.random.randint(100)  
np.random.seed(seed)
np.random.shuffle(x_data_sparse_maps)
np.random.seed(seed)
np.random.shuffle(x_data_dense_maps)
np.random.seed(seed)
np.random.shuffle(filter_x_data) 
np.random.seed(seed)
np.random.shuffle(labels)

n_values = np.max(labels) + 1
labels_oh = np.eye(n_values)[labels]
seeds = [8, 16, 32, 64, 128, 256, 1024, 2048, 4096, 8192]  

cv_list = ["Run_" + str(x) + "_CV_" + str(y) for x in range(num_runs) for y in range(num_test)]
MetaP_stat_df = pd.DataFrame(index=["AUC", "ACC" , "MCC", "Precision", "Recall", "F1"], columns=cv_list) 

In [6]:
from sklearn.metrics import roc_auc_score ,accuracy_score , matthews_corrcoef, precision_score, recall_score, f1_score
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import MinMaxScaler
from MetaP import MetaP  
import torch 

print("Starting 10 CV")
for i , repeat_seed in enumerate(seeds) : 
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=repeat_seed)
    skf_index = skf.split(x_data_dense_maps, labels) 
    
    for j , idx in enumerate(skf_index) :
        fold_num = "fold_%s" % str(j).zfill(2)
        print('#'*50 + ' repeat_seed: %s; %s ' % (repeat_seed, fold_num) + '#'*50 )

        train_index, test_index = idx    
        # train_x, test_x = x_data_sparse_maps[train_index], x_data_sparse_maps[test_index]
        
        train_x, test_x = x_data_dense_maps[train_index], x_data_dense_maps[test_index]
        #train_y, test_y = labels_oh[train_index], labels_oh[test_index]
        train_y, test_y = labels[train_index], labels[test_index]

        tree_row = train_x.shape[1]
        tree_col = train_x.shape[2]
        scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
        train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
        test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

        class DatasetLoarder(Dataset) :
            def __init__(self , X_train , y_train) :
                self.len = X_train.shape[0]
                self.x_data = torch.from_numpy(X_train).type(torch.FloatTensor)
                self.y_data = torch.from_numpy(y_train).type(torch.LongTensor)
                
            def __getitem__(self , index) :
                return self.x_data[index] , self.y_data[index]

            def __len__(self) :
                return self.len 

        train_dataset  = DatasetLoarder(train_x , train_y)  
        train_loader = DataLoader(dataset=train_dataset , batch_size=32 , num_workers=0) # num_workers 线程并行数
        test_dataset = DatasetLoarder(test_x , test_y)
        test_loader = DataLoader(dataset=test_dataset , batch_size=32 , num_workers=0 ) # num_workers 线程并行数
                
                
        def eval(model , test_loader): # test
            # loss function
            #criterion = torch.nn.CrossEntropyLoss()
            model.eval()
            true_label = [] 
            y_prob = []
            with torch.no_grad():

                for step , batch in enumerate(test_loader):
                    x, label = batch
                    
                    val_output = model(x)
                    #val_loss = criterion(val_output, label)
                    
                    true_label  = true_label + label.tolist() 
                    y_prob = y_prob + val_output.tolist()
                    
            y_prob = np.array(y_prob)
            true_label = np.array(true_label)
            return true_label , y_prob 

        def train(model , train_loader , test_loader , learn_rate , epoch ) :
            
            # loss function
            criterion = torch.nn.CrossEntropyLoss()
            # optimizer
            optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate ) # 2e-4 
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5 , gamma=0.5)
            
            for i in range(epoch):
                model.train()
                # one epoch
                for step, batch in enumerate(train_loader):
                    x, label = batch
                    output = model(x)
                    loss = criterion(output, label)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                # y_true , y_prob  = eval(model=model , test_loader=test_loader_species) 
                # y_pred = np.argmax(y_prob , axis=1)
                # #print("method = {}  test Epoch:{} ,   eval_acc:{} , eval_auc :{} ".format(method , i, round(accuracy_score(y_true, y_pred), 3) , round(roc_auc_score(y_true, y_prob, multi_class='ovo'), 3)))
                # print("method = {}  test Epoch:{} ,  eval_acc:{} ".format(method , i, round(accuracy_score(y_true, y_pred), 3)))

                 
        model = MetaP(
            image_h= train_x.shape[1],
            image_w= train_x.shape[2],
            segments = 8,
            patch_h = train_x.shape[1] ,
            patch_w = 3 ,
            dim = 48,
            depth = 1,
            num_classes = 2 ,
            expansion_factor = 1, 
        )
        print("training........ Micro Permutator model ................. ")

        train(model  , train_loader , test_loader , learn_rate=5e-4 , epoch=20)
        
        y_true , y_prob  = eval(model=model , test_loader=test_loader) 
        y_pred = np.argmax(y_prob , axis=1)
       
       
        metrics = {   
            "ACC" : round(accuracy_score(y_true, y_pred), 3),
            "Recall" : round(recall_score(y_true, y_pred , average='weighted') , 3 ) ,
            "Precision" : round(precision_score(y_true, y_pred, average='weighted') , 3) ,
            "F1"    : round(f1_score(y_true, y_pred, average='weighted') , 3) , 
            "MCC" : round(matthews_corrcoef(y_true, y_pred), 3),
            "AUC" : round(roc_auc_score(y_true, y_prob[:, 1]) , 3) ,
            "repeat_seed" : repeat_seed ,  
            "method" : "Meta_P",
            } 
        
        print("MetaP run {} ,  fold {} , metrics {}".format(i , j , metrics))

        MetaP_stat_df.loc["AUC"]["Run_" + str(i) + "_CV_" + str(j)] = metrics["AUC"]
        MetaP_stat_df.loc["ACC"]["Run_" + str(i) + "_CV_" + str(j)] = metrics["ACC"]
        MetaP_stat_df.loc["MCC"]["Run_" + str(i) + "_CV_" + str(j)] = metrics["MCC"]
        MetaP_stat_df.loc["Precision"]["Run_" + str(i) + "_CV_" + str(j)] =  metrics["Precision"]
        MetaP_stat_df.loc["Recall"]["Run_" + str(i) + "_CV_" + str(j)]= metrics["Recall"]
        MetaP_stat_df.loc["F1"]["Run_" + str(i) + "_CV_" + str(j)] =  metrics["F1"] 

        del(model) 
    #     break
    # break
                

Starting 10 CV
################################################## repeat_seed: 8; fold_00 ##################################################
training........ Micro Permutator model ................. 
MetaP run 0 ,  fold 0 , metrics {'ACC': 0.714, 'Recall': 0.714, 'Precision': 0.716, 'F1': 0.713, 'MCC': 0.429, 'AUC': 0.732, 'repeat_seed': 8, 'method': 'Meta_P'}
################################################## repeat_seed: 8; fold_01 ##################################################
training........ Micro Permutator model ................. 
MetaP run 0 ,  fold 1 , metrics {'ACC': 0.743, 'Recall': 0.743, 'Precision': 0.749, 'F1': 0.74, 'MCC': 0.49, 'AUC': 0.778, 'repeat_seed': 8, 'method': 'Meta_P'}
################################################## repeat_seed: 8; fold_02 ##################################################
training........ Micro Permutator model ................. 
MetaP run 0 ,  fold 2 , metrics {'ACC': 0.771, 'Recall': 0.771, 'Precision': 0.785, 'F1': 0.768, 'MCC': 0.

In [7]:
# results_dir = "./results/" + disease
# MetaP_stat_df.to_csv(results_dir + "/MetaP_Half_W_ALL_result.csv")
# MetaP_stat_df.mean(1).to_csv(results_dir + "/MetaP_Half_W_evaluation.csv")
# MetaP_stat_df.std(1).to_csv(results_dir + "/MetaP_Half_W_evaluation.csv", mode='a') 
print(MetaP_stat_df.mean(1))
print(MetaP_stat_df.std(1))

AUC          0.78598
ACC          0.72379
MCC          0.45335
Precision    0.72995
Recall       0.72379
F1           0.72187
dtype: float64
AUC          0.076265
ACC          0.080058
MCC          0.161878
Precision    0.082103
Recall       0.080058
F1           0.080462
dtype: float64
